In [1]:
from TorchMetricLogger import TorchMetricLogger as TML
from TorchMetricLogger import TmlMetric, TmlMetricFunction, TMLBinaryAccuracy, TMLDiceCoefficient
import torch

In [2]:
tml = TML()

p_1d = torch.ones((10, 5))
y_1d = torch.ones((10, 5))

p_1d[:, 3:] = 0

p_2d = torch.ones((10, 5, 5))
y_2d = torch.ones((10, 5, 5))

p_2d[0] = 0

w = torch.ones(10)
w[0] = 0.5

tml(
    meow=TmlMetric(p_1d, y_1d, metric_class=TMLBinaryAccuracy), 
    miau=TmlMetric(p_1d, y_1d, metric_class=TMLDiceCoefficient),
    wuff=TmlMetric(p_2d, y_2d, weights=w, metric_class=TMLDiceCoefficient)
)

In [3]:
tml.metrics["meow"].reduce()

0.6

In [4]:
tml.metrics["wuff"].partial

[[array([0.03846154, 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ],
        dtype=float32),
  array([0.5, 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ], dtype=float32)]]

In [5]:
tml.metrics["wuff"].reduce()

0.94939274

In [6]:
tml.metrics["miau"].reduce()

0.7777777910232544

In [7]:
tml = TML()

In [8]:
# take the (weighted) mean, usefull for loss functions
class TMLF1(TmlMetricFunction):
    def __call__(self, metric):
        if metric.weights is None:
            metric.weights = np.ones(metric.predictions.size)
    
        super().__call__(metric)
        
    def calculate(self, metric):
        tp = np.sum(((metric.gold_labels >= 0.5) * (metric.predictions >= 0.5)) * metric.weights)
        fp = np.sum(((metric.gold_labels < 0.5) * (metric.predictions >= 0.5)) * metric.weights)
        fn = np.sum(((metric.gold_labels >= 0.5) * (metric.predictions < 0.5)) * metric.weights)
        
        return [tp, fp, fn]

    def reduction_function(self):
        tp = self.partial[:, 0]
        fp = self.partial[:, 1]
        fn = self.partial[:, 2]

        return np.mean(tp / (tp + (fp + fn)/2) )

In [9]:
import numpy as np

p = np.array([1,1,1,1,0,0])
l = np.array([1,1,1,1,1,1])

tml(
    meow=TmlMetric(p, l, metric_class=TMLF1)
)

tml.on_batch_end()

[4.] [2.] [0.]


In [10]:
tml.metrics["meow"].partial

[]

In [11]:
tml.metrics["meow"].history

[0.8]

In [12]:
from sklearn.metrics import f1_score

In [13]:
f1_score(l, p)

0.8